## Deliverable 2. Create a Customer Travel Destinations Map.

In [67]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# To add NaN values
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [68]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,76.96,66,40,15.03,scattered clouds
1,1,Lebu,CL,-37.6167,-73.6500,53.38,85,100,18.25,overcast clouds
2,2,Antanifotsy,MG,-19.6500,47.3167,68.67,33,3,4.34,clear sky
3,3,Butaritari,KI,3.0707,172.7902,82.36,74,78,16.24,broken clouds
4,4,Torbay,CA,47.6666,-52.7314,74.75,74,75,13.80,broken clouds


In [69]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [71]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,76.96,66,40,15.03,scattered clouds
3,3,Butaritari,KI,3.0707,172.7902,82.36,74,78,16.24,broken clouds
5,5,Rikitea,PF,-23.1203,-134.9692,75.33,78,25,17.69,scattered clouds
9,9,Saskylakh,RU,71.9167,114.0833,77.31,43,100,7.23,overcast clouds
10,10,Labuhan,ID,-6.8844,112.2051,79.43,68,90,8.97,overcast clouds
19,19,Grand Gaube,MU,-20.0064,57.6608,75.38,64,4,5.99,clear sky
21,21,Cap Malheureux,MU,-19.9842,57.6142,75.47,64,4,5.99,clear sky
25,25,Hasaki,JP,35.7333,140.8333,82.29,90,100,14.56,overcast clouds
37,37,Nouadhibou,MR,20.9310,-17.0347,76.98,83,7,18.41,clear sky
38,38,Kudat,MY,6.8837,116.8477,78.73,94,20,8.05,few clouds


In [72]:
# 4a. Determine if there are any empty rows. (3 null values were dropped)
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [73]:
# Experiment with adding NaN values to any columns.
# Code taken from: https://sparkbyexamples.com/pandas/pandas-replace-blank-values-with-nan/
test_df = preferred_cities_df.replace(r'^\s*$', np.nan, regex=True)

In [74]:
# Check for all NaN values. There are still only 3.
test_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [75]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [76]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,76.96,scattered clouds,-9.8000,-139.0333,
3,Butaritari,KI,82.36,broken clouds,3.0707,172.7902,
5,Rikitea,PF,75.33,scattered clouds,-23.1203,-134.9692,
9,Saskylakh,RU,77.31,overcast clouds,71.9167,114.0833,
10,Labuhan,ID,79.43,overcast clouds,-6.8844,112.2051,
19,Grand Gaube,MU,75.38,clear sky,-20.0064,57.6608,
21,Cap Malheureux,MU,75.47,clear sky,-19.9842,57.6142,
25,Hasaki,JP,82.29,overcast clouds,35.7333,140.8333,
37,Nouadhibou,MR,76.98,clear sky,20.9310,-17.0347,
38,Kudat,MY,78.73,few clouds,6.8837,116.8477,


In [77]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [78]:
# 7. Drop the rows where there is no Hotel Name. (1st attempt)
clean_hotel_df = hotel_df.dropna()

In [79]:
# Confirm hotel_df DataFrame appears similar to the screenshot in the challenge.
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,76.96,scattered clouds,-9.8000,-139.0333,Villa Enata
3,Butaritari,KI,82.36,broken clouds,3.0707,172.7902,Isles Sunset Lodge
5,Rikitea,PF,75.33,scattered clouds,-23.1203,-134.9692,People ThankYou
9,Saskylakh,RU,77.31,overcast clouds,71.9167,114.0833,
10,Labuhan,ID,79.43,overcast clouds,-6.8844,112.2051,DoubleFAG
19,Grand Gaube,MU,75.38,clear sky,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
21,Cap Malheureux,MU,75.47,clear sky,-19.9842,57.6142,Pereybere Hotel & Spa
25,Hasaki,JP,82.29,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
37,Nouadhibou,MR,76.98,clear sky,20.9310,-17.0347,El Medina
38,Kudat,MY,78.73,few clouds,6.8837,116.8477,Dream Garden Hotel


In [80]:
# Investigate empty hotel name in row 9.
clean_hotel_df.iloc[3,6]

# Unsure why this remains an empty string

''

In [81]:
# Replace empty hotel name rows with NaN values.
# Code taken from: https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,76.96,scattered clouds,-9.8000,-139.0333,Villa Enata
3,Butaritari,KI,82.36,broken clouds,3.0707,172.7902,Isles Sunset Lodge
5,Rikitea,PF,75.33,scattered clouds,-23.1203,-134.9692,People ThankYou
9,Saskylakh,RU,77.31,overcast clouds,71.9167,114.0833,NaN
10,Labuhan,ID,79.43,overcast clouds,-6.8844,112.2051,DoubleFAG
19,Grand Gaube,MU,75.38,clear sky,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
21,Cap Malheureux,MU,75.47,clear sky,-19.9842,57.6142,Pereybere Hotel & Spa
25,Hasaki,JP,82.29,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
37,Nouadhibou,MR,76.98,clear sky,20.9310,-17.0347,El Medina
38,Kudat,MY,78.73,few clouds,6.8837,116.8477,Dream Garden Hotel


In [84]:
# Check for NaN values.
clean_hotel_df.isnull().sum()

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             15
dtype: int64

In [86]:
# 7. Drop the rows where there is no Hotel Name. (2nd attempt)
# Code taken from: https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
clean_hotel_df.dropna(subset=['Hotel Name'], inplace=True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,76.96,scattered clouds,-9.8000,-139.0333,Villa Enata
3,Butaritari,KI,82.36,broken clouds,3.0707,172.7902,Isles Sunset Lodge
5,Rikitea,PF,75.33,scattered clouds,-23.1203,-134.9692,People ThankYou
10,Labuhan,ID,79.43,overcast clouds,-6.8844,112.2051,DoubleFAG
19,Grand Gaube,MU,75.38,clear sky,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
21,Cap Malheureux,MU,75.47,clear sky,-19.9842,57.6142,Pereybere Hotel & Spa
25,Hasaki,JP,82.29,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
37,Nouadhibou,MR,76.98,clear sky,20.9310,-17.0347,El Medina
38,Kudat,MY,78.73,few clouds,6.8837,116.8477,Dream Garden Hotel
39,Lazurne,UA,76.93,light rain,46.0840,32.5295,New Manna Hostel


In [87]:
# Confirm that all null values were dropped.
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [88]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [92]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt>{Current Description}
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [93]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))